In [22]:
import pandas as pd
import json
import os

path_labels_lifecycle = 'labels_lifecycle.json'
folder = os.path.join("..", "0_dataset")
log_files = [['label_syslog_disk_wipe.csv', 1],
               ['label_syslog_end_point_dos.csv', 4],
               ['label_syslog_mirai.csv', 0],
               ['label_syslog_ransomware.csv', 2],
               ['label_syslog_resource_hijacking.csv', 3]]
merged_list = ['results_T.csv', 'results_A.csv', 'results_L.csv', 'results_A+L.csv', 'results_A+T.csv', 'results_L+T.csv', 'results_A+L+T.csv']
# merged_list = ['results_A+L+T.csv']
folder_lifecycle = 'lifecycle'

Get start and end time for each scenario

In [20]:
time_list = []
for file, label in log_files:
    df_tmp = pd.read_csv(os.path.join(folder, 'syslog', file))
    time_list.append([df_tmp['Timestamp'].iloc[0], df_tmp['Timestamp'].iloc[-1], label])
    
print(time_list)

[[1673433290.0, 1673434411.0, 1], [1673439558.0, 1673440994.0, 4], [1673430856.0, 1673432522.0, 0], [1673435423.0, 1673436706.0, 2], [1673437131.0, 1673438383.0, 3]]


Sequences Collecting

In [29]:
df_test = pd.DataFrame(columns=['lifecycle', 'Label'])
df_test['lifecycle'] = df_test['lifecycle'].astype('object')
df_train = pd.DataFrame(columns=['lifecycle', 'Label'])
df_train['lifecycle'] = df_train['lifecycle'].astype('object')
df_test['lifecycle'] = df_test['lifecycle'].astype('object')
df_pattern = pd.DataFrame(columns=['lifecycle', 'Label'])
df_pattern['lifecycle'] = df_pattern['lifecycle'].astype('object')

# get label
with open(path_labels_lifecycle, "r") as f:
    data = json.load(f)
    for label, name in data[1:]:
        isfind = False
        for start, end, label_time in time_list:
            # print(f'label:{label}, label_time:{label_time}')
            if int(label) == label_time:
                t_start = start
                t_end = end
                isfind = True
                print(f'\nlabel:{label} t_start:{t_start}, t_end:{t_end}')
                break
        
        if isfind == False:
            continue
        
        # get merged sequences
        def preprocess_seq(df_tmp, label, ispred):
            if ispred:
                seq = df_tmp.loc[((df_tmp['TIMESTAMP'] >= t_start) & (df_tmp['TIMESTAMP'] <= t_end)), 'pred'].tolist()
            else:
                seq = df_tmp.loc[((df_tmp['TIMESTAMP'] >= t_start) & (df_tmp['TIMESTAMP'] <= t_end)), 'label'].tolist()
            while seq and seq[0] == 0:
                seq.pop(0)
            while seq and seq[-1] == 0:
                seq.pop()
            return pd.Series([seq, label], index=['lifecycle', 'Label'])
        
        for merged_res in merged_list:
            df_tmp = pd.read_csv(os.path.join(folder, folder_lifecycle, merged_res))
            df_test.loc[len(df_test)] = preprocess_seq(df_tmp, label, False)
            df_train.loc[len(df_train)] = preprocess_seq(df_tmp, label, True)
            
            df_show = df_tmp.loc[((df_tmp['TIMESTAMP'] >= t_start) & (df_tmp['TIMESTAMP'] <= t_end)), 'label']
            print(f'{merged_res}: start:{t_start} end:{t_end} len:{len(df_show)}')
            
            if 'A+L+T' in merged_res:
                df_pattern.loc[len(df_pattern)] = preprocess_seq(df_tmp, label, False)

print(df_test.info)
print(df_train.info)
print(df_pattern.info)
df_test.to_csv(os.path.join(folder, folder_lifecycle, 'lifecycle_dataset_test_original.csv'), encoding='utf-8', index=False)
df_train.to_csv(os.path.join(folder, folder_lifecycle, 'lifecycle_dataset_train_original.csv'), encoding='utf-8', index=False)
df_pattern.to_csv(os.path.join(folder, folder_lifecycle, 'lifecycle_pattern.csv'), encoding='utf-8', index=False)


label:0 t_start:1673430856.0, t_end:1673432522.0
results_T.csv: start:1673430856.0 end:1673432522.0 len:1656
results_A.csv: start:1673430856.0 end:1673432522.0 len:1667
results_L.csv: start:1673430856.0 end:1673432522.0 len:1667
results_A+L.csv: start:1673430856.0 end:1673432522.0 len:1667
results_A+T.csv: start:1673430856.0 end:1673432522.0 len:1667
results_L+T.csv: start:1673430856.0 end:1673432522.0 len:1667
results_A+L+T.csv: start:1673430856.0 end:1673432522.0 len:1667

label:1 t_start:1673433290.0, t_end:1673434411.0
results_T.csv: start:1673433290.0 end:1673434411.0 len:945
results_A.csv: start:1673433290.0 end:1673434411.0 len:1122
results_L.csv: start:1673433290.0 end:1673434411.0 len:1122
results_A+L.csv: start:1673433290.0 end:1673434411.0 len:1122
results_A+T.csv: start:1673433290.0 end:1673434411.0 len:1122
results_L+T.csv: start:1673433290.0 end:1673434411.0 len:1122
results_A+L+T.csv: start:1673433290.0 end:1673434411.0 len:1122

label:2 t_start:1673435423.0, t_end:1673

Technique Sequence Generation

In [38]:
import ast
import random

# data_type = 'train'
data_type = 'test'
df = pd.read_csv(os.path.join(folder, folder_lifecycle, 'lifecycle_dataset_'+data_type+'_original.csv'))

times_generating = 12
len_rand = 5


df_new = pd.DataFrame(columns=['lifecycle', 'Label'])
max_len = 0
for i in range(times_generating):
    for j, item in enumerate(df['lifecycle']):
        list_new = []
        lifecycle = ast.literal_eval(item)
        index = 0
        while index < len(lifecycle):
            if lifecycle[index] != 0:
                list_new.append(lifecycle[index])
                index += 1
            else:
                index_new = index
                while index_new < len(lifecycle) and lifecycle[index_new] == 0:
                    index_new += 1
                len_0 = index_new - index
                num_rand = random.randint(max(0, len_0 - len_rand), len_0 + len_rand)
                list_new.extend([0] * num_rand)
                index = index_new

        max_len = max(max_len, len(list_new))
        # print(len(lifecycle))
        # print(len(list_new))
        # print('----')
        df_new.loc[len(df_new)] = pd.Series([list_new, df['Label'][j]], index=['lifecycle', 'Label'])

df = pd.concat([df, df_new], ignore_index=True)
df.to_csv(os.path.join(folder, folder_lifecycle, 'lifecycle_dataset_'+data_type+'.csv'), encoding='utf-8', index=False)

print(f'max sequence length: {max_len}')
for label in sorted(df['Label'].unique()):
    len_label = len(df[df['Label'] == label])
    print(f'label {label} total data: {len_label}')
print(df)

max sequence length: 2093
label 0 total data: 91
label 1 total data: 91
label 2 total data: 91
label 3 total data: 91
label 4 total data: 91
                                             lifecycle Label
0    [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, ...     0
1    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...     0
2    [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...     0
3    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...     0
4    [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, ...     0
..                                                 ...   ...
450  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...     4
451  [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, ...     4
452  [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...     4
453  [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...     4
454  [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, ...     4

[455 rows x 2 columns]


Common Techniques (LCS DP)

In [18]:
import ast
from itertools import groupby

df = pd.read_csv(os.path.join(folder, folder_lifecycle, 'lifecycle_dataset_original.csv'))
target_label = 0
remaining_labels = [1, 2, 3, 4]
target = df[df['Label'] == target_label].values[0][0]
df_remain = df[df['Label'] != target_label]
res = {}


def preprocess(seq):
    '''
    seq: str type, but content is list
    '''
    seq = ast.literal_eval(seq)
    seq = [value for value in seq if value != 0]
    seq = [key for key, _group in groupby(seq)]
    
    return seq

def lcs(X, Y):
    '''
    X, Y: both are list
    '''
    m = len(X)
    n = len(Y)
    L = [[0] * (n + 1) for i in range(m + 1)]
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])
    
    index = L[m][n]
    common_seq = [None] * index
    i = m
    j = n
    while i > 0 and j > 0:
        if X[i - 1] == Y[j - 1]:
            common_seq[index - 1] = X[i - 1]
            i -= 1
            j -= 1
            index -= 1
        elif L[i - 1][j] > L[i][j - 1]:
            i -= 1
        else:
            j -= 1
    
    return common_seq


target = preprocess(target)
for seq, label in df_remain.values:
    seq = preprocess(seq)
    
    common_techs = lcs(seq, target)
    print(f'lifecycle {target_label} and lifecycle {label}:')
    print(f'target:\n{target}')
    print(f'seq:\n{seq}')
    print(f'common techs:\n{common_techs}\n')

lifecycle 0 and lifecycle 1:
target:
[1, 14, 1, 4, 1, 2, 14, 1, 11, 1, 13, 15, 13]
seq:
[1, 14, 1, 14, 1, 14, 1, 14, 1, 14, 4, 2, 4, 1, 2, 1, 2, 1, 4, 1, 2, 4, 1, 4, 1, 4, 2, 1, 14, 1, 14]
common techs:
[1, 14, 1, 4, 1, 2, 14, 1]

lifecycle 0 and lifecycle 2:
target:
[1, 14, 1, 4, 1, 2, 14, 1, 11, 1, 13, 15, 13]
seq:
[1, 14, 1, 2, 4, 13, 1, 13, 1, 13, 1, 13, 2, 1, 13, 1, 13, 1, 13, 1, 13, 8, 13, 1, 14, 13, 1, 13, 1, 13, 1, 13, 1, 13, 15, 13]
common techs:
[1, 14, 1, 4, 1, 2, 14, 1, 1, 13, 15, 13]

lifecycle 0 and lifecycle 3:
target:
[1, 14, 1, 4, 1, 2, 14, 1, 11, 1, 13, 15, 13]
seq:
[1, 14, 1, 2, 1, 4, 2, 1, 4, 1, 4, 1, 4, 1, 14, 1, 2, 1, 14, 1]
common techs:
[1, 14, 1, 4, 1, 2, 14, 1]

lifecycle 0 and lifecycle 4:
target:
[1, 14, 1, 4, 1, 2, 14, 1, 11, 1, 13, 15, 13]
seq:
[1, 8, 14, 2, 4, 2, 4, 2, 4, 2, 4, 14, 15, 17, 13, 1, 13, 1, 13]
common techs:
[1, 14, 4, 2, 14, 1, 1, 13]



Edit Distance Comparison

In [16]:
import os
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv(os.path.join(folder, folder_lifecycle, 'lifecycle_dataset.csv'))
pattern = pd.read_csv(os.path.join(folder, folder_lifecycle, 'lifecycle_pattern.csv'))
print(df.info)


X = df['lifecycle']
X = X.to_numpy()
y = df['Label']
y = y.to_numpy()
y = y.reshape(-1)


def get_similarity(A, B):
    '''
    use levenshtein distance to compute the similarity
    '''
    m = len(A)
    n = len(B)
    D = [[0] * (n + 1) for i in range(m + 1)]

    for i in range(m + 1):
        D[i][0] = i
    for j in range(n + 1):
        D[0][j] = j

    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if A[i - 1] == B[j - 1]:
                D[i][j] = D[i - 1][j - 1]
            else:
                D[i][j] = min(D[i - 1][j] + 1, D[i][j - 1] + 1, D[i - 1][j - 1] + 1)

    similarity = 1.0 - (float(D[m][n]) / float(max(len(A), len(B))))

    return similarity

for i in range(len(X)):
    get_similarity(X[i],)

<bound method DataFrame.info of                                               lifecycle  Label
0     [1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...      0
1     [1, 0, 0, 1, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 14...      1
2     [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, ...      2
3     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...      3
4     [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, ...      4
...                                                 ...    ...
2000  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...      0
2001  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 14,...      1
2002  [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...      2
2003  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...      3
2004  [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...      4

[2005 rows x 2 columns]>


TypeError: get_similarity() missing 1 required positional argument: 'B'

3 approaches comparison

In [17]:
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt
import math

evaluation = []
evaluation[name]['accuracy'].append(accuracy_score(y_test, y_hat,))
evaluation[name]['precision'].append(precision_score(y_test, y_hat, average='weighted',zero_division=0))
evaluation[name]['recall'].append(recall_score(y_test, y_hat, average='weighted', zero_division=0))
evaluation[name]['f1_score'].append(f1_score(y_test, y_hat, average='weighted', zero_division=0))

def round_off(value, n):
    return math.floor(value * (10 ** n)) / float(10 ** n) 

models = [name.replace('_', '\n') for name in evaluation]
result = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}
for name in evaluation:
    for key, value in evaluation[name].items():
        result[key].append(round_off(mean(value), 3))
        
        if name == 'Bi-GRU':
            print(f'metric {key}: {round_off(mean(value), 3)}')

width = 0.2
x = np.arange(len(models))
plt.figure(figsize=(20, 10))
plt.bar(x, result['accuracy'], width, label='accuracy')
plt.bar(x+width, result['precision'], width, label='precision')
plt.bar(x+2*width, result['recall'], width, label='recall')
bar = plt.bar(x+3*width, result['f1_score'], width, label='f1_score')
plt.bar_label(bar, label_type='edge', fontsize=20)
plt.title('Approaches Performance Comparison', fontsize=30)
plt.xticks(x+1.5*width, models)
plt.xlabel('model', fontsize=25)
plt.ylabel('value', fontsize=22)
plt.rcParams.update({
    'xtick.labelsize': 18,
    'ytick.labelsize': 16,
})
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=20)
plt.ylim(0, 1.05)
plt.show()

NameError: name 'models' is not defined